# Lab 3 - Exploring Joining strategies

In this lab, we will investigate the joining of the property data from 2004-2015 to the water quality data.  First, we need to 

1. Understand the big picture and determine the key(s) used to join tables.
2. Understand the relationship between tables (one-to-one; one-to-many; many-to-many), and
3. Pick a join type and investigate key mismatches.

## Problem 1 - Understanding the big picture and tables keys

<img src="./img/joining_tables.png" width="800">

**Task.**  The picture shown above provides a high-level view of the joining process. Remember that our goal is to explain changes in water quality in various lakes over time.  What do you think the structure of the final table should be?  One row per what? Describe some examples of the sort of columns you would want present in this file.

> <font color="orange"> We want the final table that contains the water quality for each year from 2004-2007. we want the columns added from each year from our previous joins added into our final data and there should only be one row of information for each of the lake for that specific year . it should be water quality information, with taxation information, ending with lake centroids</font>

## Problem 2 - Understanding the big picture and tables keys

**Tasks.**

1. Use `!head` to inspect both the water quality and XREF files to determine the structure, namely the column separator.
2. Read in each of these files and suggest the columns that will be used to join the tables.
3. To understand the relationship (one-to-one; one-to-many; many-to-many) between tables, perform aggregation on each table to determine if there is one or many keys per row.
4. Based on the results of the last task, suggest a join type and justify your response.
5. For each table, create query that results in a column with one unique key per row.
6. Perform the join suggested in **4.** and investigate any mismatches.  Document your findings and suggest necessary remedies.

In [1]:
!head path ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt

head: cannot open 'path' for reading: No such file or directory
==> ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt <==
7105	Lakes Monitoring	Zumbra Lake	Victoria	Carver	10004100-01	Mississippi River - Twin Cities	Minnehaha Creek WD	1	2002-10-14	0:00	2002-10-14	0:00	7													m	~	0.012	Approved	mg/L	-93.66768906	44.88381717.88381717177d			0.9	Approved	m		0.08	Approved	mg/L	-93.90614956	44.783110271144.96339695566363 good & 5 bad		2.7	Approved	m		0.02	Approved	mg/L	-93.63778927	44.83812233	mg/L	-92.97144206	44.944613695itability_QUALIFIER	Recreational_Suitability_Units	Secchi_Depth_RESULT_SIGN	Secchi_Depth_RESULT	Secchi_Depth_QUALIFIER	Secchi_Depth_Units	Total_Phosphorus_RESULT_SIGN	Total_Phosphorus_RESULT	Total_Phosphorus_QUALIFIER	Total_Phosphorus_Units	longitude	latitude92	44.86790536a	Carver	10001900-01	Lower Minnesota River	Carver County WMO	1	1987-09-15	9:05	1987-09-15	9:05	17			0-4 Categorical Calculated Seasonally: 4 good & 0 bad			1-5 Categorical: 

In [2]:
!head path ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt

head: cannot open 'path' for reading: No such file or directory
==> ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt <==
Parcel_PIN	Monit_MAP_CODE1	Monit_SITE_CODE	Monit_LAKE_SITE	Distance_Parcel_Monitoring_Site_meters	Lake_Hydroid	Distance_Parcel_Lake_meters	centroid_long	centroid_lat	Parcel_pkey
	19007900-01	19007900	1	2815.4927104148851	110517277058	2571.5267922258381	-93.11451	44.94283	2163034
	19007900-01	19007900	1	2753.4746875312162	110517277058	2515.3738022144425	-93.11539	44.94234	2163035
	19007900-01	19007900	1	2748.8106650822624	110517277058	2511.9249588634261	-93.11556	44.94231	2163036
	19007900-01	19007900	1	2738.7500558227048	110517277058	2502.9916401143814	-93.11572	44.94223	2163037
	19007900-01	19007900	1	2700.5414124735426	110517277058	2465.2062339003005	-93.1158	44.94189	2163038
	19007900-01	19007900	1	2890.2197211400312	110517277058	2631.4612562849106	-93.11218	44.94325	2163039
	19007900-01	19007900	1	3338.9566080000855	1105172770

In [3]:
from glob import glob as original_glob
from more_pyspark import to_pandas
from composable import pipeable
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').getOrCreate()

your 131072x1 screen size is bogus. expect trouble


22/11/29 19:21:36 WARN Utils: Your hostname, lu4543hm221 resolves to a loopback address: 127.0.1.1; using 172.19.14.158 instead (on interface eth0)
22/11/29 19:21:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/29 19:21:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
lakepath = './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/mces_lakes_1999_2014.txt'
refpath = './data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt'
quality = spark.read.csv(lakepath, header=True, sep='\t')
ref = spark.read.csv(refpath, header=True, sep='\t')
quality

DataFrame[PROJECT_ID: string, DATA_SET_TITLE: string, LAKE_NAME: string, CITY: string, COUNTY: string, DNR_ID_Site_Number: string, MAJOR_WATERSHED: string, WATER_PLANNING_AUTHORITY: string, LAKE_SITE_NUMBER: string, START_DATE: string, START_HOURMIN24: string, END_DATE: string, END_HOURMIN24: string, SAMPLE_DEPTH_IN_METERS: string, Seasonal_Lake_Grade_RESULT: string, Seasonal_Lake_Grade_QUALIFIER: string, Seasonal_Lake_Grade_Units: string, Physical_Condition_RESULT: string, Physical_Condition_QUALIFIER: string, Physical_Condition_Units: string, Recreational_Suitability_RESULT: string, Recreational_Suitability_QUALIFIER: string, Recreational_Suitability_Units: string, Secchi_Depth_RESULT_SIGN: string, Secchi_Depth_RESULT: string, Secchi_Depth_QUALIFIER: string, Secchi_Depth_Units: string, Total_Phosphorus_RESULT_SIGN: string, Total_Phosphorus_RESULT: string, Total_Phosphorus_QUALIFIER: string, Total_Phosphorus_Units: string, longitude: string, latitude: string]

In [5]:
quality.toPandas().head(3)

22/11/29 19:22:02 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,...,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-04-16,...,None,1,Approved,m,None,0.156,Approved,mg/L,-92.97171054,45.01655642
1,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-01,...,None,None,None,m,None,None,None,mg/L,-92.97171054,45.01655642
2,7108,Citizen Assisted Monitoring Program (CAMP) for...,Acorn Lake,Oakdale,Washington,82010200-01,Lower St. Croix River,Valley Branch WD,1,2006-05-02,...,None,0.66,Approved,m,None,0.107,Approved,mg/L,-92.97171054,45.01655642


In [6]:
ref.toPandas().head(3)

,Parcel_PIN,Monit_MAP_CODE1,Monit_SITE_CODE,Monit_LAKE_SITE,Distance_Parcel_Monitoring_Site_meters,Lake_Hydroid,Distance_Parcel_Lake_meters,centroid_long,centroid_lat,Parcel_pkey
0,None,19007900-01,19007900,1,2815.4927104148851,110517277058,2571.5267922258381,-93.11451,44.94283,2163034
1,None,19007900-01,19007900,1,2753.4746875312162,110517277058,2515.3738022144425,-93.11539,44.94234,2163035
2,None,19007900-01,19007900,1,2748.8106650822624,110517277058,2511.9249588634261,-93.11556,44.94231,2163036


In [7]:
from pyspark.sql.functions import col
refdist = (ref
    .groupBy('Monit_MAP_CODE1')
    .count()
)
refdist.toPandas()

,Monit_MAP_CODE1,count
0,19008800-01,17008
1,10010300-01,3209
2,82051400-01,513
3,62007300-01,3364
4,82005600-01,505
...,...,...
435,82007400-03,306
436,27017500-01,3400
437,02000600-01,3509
438,27009300-01,2238


In [8]:
lakequality = (quality
    .groupBy('DNR_ID_SITE_NUMBER')
    .count()
    .where(col('count')>1)
)
lakequality.toPandas()

,DNR_ID_SITE_NUMBER,count
0,82005600-01,84
1,82001100-01,85
2,02013300-01,182
3,10008400-01,166
4,82016600-01,143
...,...,...
332,82012200-01,287
333,62003600-01,52
334,99002487-01,58
335,02008000-01,141


In [9]:
qualdist = quality.drop_duplicates(['DNR_ID_Site_Number'])
qualdist.toPandas()

,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,...,Secchi_Depth_RESULT_SIGN,Secchi_Depth_RESULT,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude
0,7105,Lakes Monitoring,Otter Lake,Lino Lakes,Anoka,02000300-01,Mississippi River - Twin Cities,Rice Creek WD,1,1980-06-17,...,None,2.8,Approved,m,None,0.04,Approved,mg/L,-93.03887311,45.12456438
1,7105,Lakes Monitoring,Peltier Lake,Lino Lakes,Anoka,02000400-01,Mississippi River - Twin Cities,Rice Creek WD,1,1983-05-01,...,None,None,None,m,None,None,None,mg/L,-93.05836626,45.17785408
2,7105,Lakes Monitoring,George Watch Lake,Lino Lakes,Anoka,02000500-01,Mississippi River - Twin Cities,Rice Creek WD,1,1981-05-01,...,None,None,None,m,None,None,None,mg/L,-93.08905468,45.17640481
3,7105,Lakes Monitoring,Centerville Lake,Lino Lakes,Anoka,02000600-01,Mississippi River - Twin Cities,Rice Creek WD,1,1980-05-01,...,None,None,None,m,None,None,None,mg/L,-93.07493329,45.16261181
4,7105,Lakes Monitoring,Marshan Lake,Lino Lakes,Anoka,02000700-01,Mississippi River - Twin Cities,Rice Creek WD,1,1981-05-01,...,None,None,None,m,None,None,None,mg/L,-93.11032905,45.16844769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,7108,Citizen Assisted Monitoring Program (CAMP) for...,Legion Pond,Lake Elmo,Washington,82046200-01,Lower St. Croix River,Valley Branch WD,1,2005-04-17,...,None,1.2,Approved,m,None,0.051,Approved,mg/L,-92.87497498,44.98916957
345,7108,Citizen Assisted Monitoring Program (CAMP) for...,Mergens Pond,West Lakeland Township,Washington,82048200-01,Lower St. Croix River,Valley Branch WD,1,2000-05-01,...,None,None,None,m,None,None,None,mg/L,-92.82652345,44.96768212
346,7108,Citizen Assisted Monitoring Program (CAMP) for...,West Lakeland Storage Site,West Lakeland Township,Washington,82048800-01,Lower St. Croix River,Valley Branch WD,1,2000-05-19,...,None,1.5,Approved,m,None,0.04,Approved,mg/L,-92.84742244,44.96339695
347,7108,Citizen Assisted Monitoring Program (CAMP) for...,Rest Area Pond,West Lakeland Township,Washington,82051400-01,Lower St. Croix River,Valley Branch WD,1,2006-04-20,...,None,0.78,Approved,m,None,0.055,Approved,mg/L,-92.81688162,44.95345401


In [10]:
refix = refdist.drop_duplicates(['Monit_MAP_CODE1'])
refix.toPandas()

,Monit_MAP_CODE1,count
0,19008800-01,17008
1,10010300-01,3209
2,82051400-01,513
3,62007300-01,3364
4,82005600-01,505
...,...,...
435,82007400-03,306
436,27017500-01,3400
437,02000600-01,3509
438,27009300-01,2238


In [11]:
joinedp2 = qualdist.join(refix, qualdist.DNR_ID_Site_Number == refix.Monit_MAP_CODE1, "outer").toPandas()
joinedp2

,PROJECT_ID,DATA_SET_TITLE,LAKE_NAME,CITY,COUNTY,DNR_ID_Site_Number,MAJOR_WATERSHED,WATER_PLANNING_AUTHORITY,LAKE_SITE_NUMBER,START_DATE,...,Secchi_Depth_QUALIFIER,Secchi_Depth_Units,Total_Phosphorus_RESULT_SIGN,Total_Phosphorus_RESULT,Total_Phosphorus_QUALIFIER,Total_Phosphorus_Units,longitude,latitude,Monit_MAP_CODE1,count
0,7105,Lakes Monitoring,Otter Lake,Lino Lakes,Anoka,02000300-01,Mississippi River - Twin Cities,Rice Creek WD,1,1980-06-17,...,Approved,m,None,0.04,Approved,mg/L,-93.03887311,45.12456438,02000300-01,4790.0
1,7105,Lakes Monitoring,Peltier Lake,Lino Lakes,Anoka,02000400-01,Mississippi River - Twin Cities,Rice Creek WD,1,1983-05-01,...,None,m,None,None,None,mg/L,-93.05836626,45.17785408,02000400-01,5071.0
2,7105,Lakes Monitoring,George Watch Lake,Lino Lakes,Anoka,02000500-01,Mississippi River - Twin Cities,Rice Creek WD,1,1981-05-01,...,None,m,None,None,None,mg/L,-93.08905468,45.17640481,02000500-01,4571.0
3,7105,Lakes Monitoring,Centerville Lake,Lino Lakes,Anoka,02000600-01,Mississippi River - Twin Cities,Rice Creek WD,1,1980-05-01,...,None,m,None,None,None,mg/L,-93.07493329,45.16261181,02000600-01,3509.0
4,7105,Lakes Monitoring,Marshan Lake,Lino Lakes,Anoka,02000700-01,Mississippi River - Twin Cities,Rice Creek WD,1,1981-05-01,...,None,m,None,None,None,mg/L,-93.11032905,45.16844769,02000700-01,3709.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,7108,Citizen Assisted Monitoring Program (CAMP) for...,Mergens Pond,West Lakeland Township,Washington,82048200-01,Lower St. Croix River,Valley Branch WD,1,2000-05-01,...,None,m,None,None,None,mg/L,-92.82652345,44.96768212,82048200-01,1442.0
441,7108,Citizen Assisted Monitoring Program (CAMP) for...,West Lakeland Storage Site,West Lakeland Township,Washington,82048800-01,Lower St. Croix River,Valley Branch WD,1,2000-05-19,...,Approved,m,None,0.04,Approved,mg/L,-92.84742244,44.96339695,82048800-01,417.0
442,7108,Citizen Assisted Monitoring Program (CAMP) for...,Rest Area Pond,West Lakeland Township,Washington,82051400-01,Lower St. Croix River,Valley Branch WD,1,2006-04-20,...,Approved,m,None,0.055,Approved,mg/L,-92.81688162,44.95345401,82051400-01,513.0
443,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,99001184-02,423.0


In [17]:
# Place your code/thoughts in one or more code/markdown cells, respectively.

## Problem 3 - Understanding the big picture and tables keys

<img src="./img/joining_tables.png" width="800">

**Tasks.**

1. We will be joining the XREF file to the parcel files using the centroid lats and longs.  Use `!head` to inspect both the XREF and parcel files to verify that these columns are floating point numbers.  
2. Remember that [floating point number](https://en.wikipedia.org/wiki/Floating-point_arithmetic) are approximations that **are not guaranteed to be exactly the same** on all machines, or even on the same machine at different times.  Explain why (A) joining on floating point numbers is problematic and (B) why we would want to keep these columns as strings when joining tables.
3. To understand the relationship (one-to-one; one-to-many; many-to-many) between tables, perform aggregation on each table to determine if there is one or many keys per row.
4. Based on the results of the last task, suggest a join type and justify your response.
5. For each table, create query that results in a column with one unique key per row.
6. Perform the join suggested in **4.** and investigate any mismatches.  Document your findings and suggest necessary remedies.

In [12]:
!head path ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt

head: cannot open 'path' for reading: No such file or directory
==> ./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/Parcel_Lake_Monitoring_Site_Xref.txt <==
Parcel_PIN	Monit_MAP_CODE1	Monit_SITE_CODE	Monit_LAKE_SITE	Distance_Parcel_Monitoring_Site_meters	Lake_Hydroid	Distance_Parcel_Lake_meters	centroid_long	centroid_lat	Parcel_pkey
	19007900-01	19007900	1	2815.4927104148851	110517277058	2571.5267922258381	-93.11451	44.94283	2163034
	19007900-01	19007900	1	2753.4746875312162	110517277058	2515.3738022144425	-93.11539	44.94234	2163035
	19007900-01	19007900	1	2748.8106650822624	110517277058	2511.9249588634261	-93.11556	44.94231	2163036
	19007900-01	19007900	1	2738.7500558227048	110517277058	2502.9916401143814	-93.11572	44.94223	2163037
	19007900-01	19007900	1	2700.5414124735426	110517277058	2465.2062339003005	-93.1158	44.94189	2163038
	19007900-01	19007900	1	2890.2197211400312	110517277058	2631.4612562849106	-93.11218	44.94325	2163039
	19007900-01	19007900	1	3338.9566080000855	1105172770

In [12]:
from glob import glob
from composable.sequence import reduce
from composable.strict import map, filter

In [13]:
me= refix.distinct()
me.count()

440

In [14]:
parcel_files1 = sorted(glob('./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/20[456789]*parcel*.txt'))
parcel_files2 = sorted(glob('./data/MinneMUDAC_raw_files/MinneMUDAC_raw_files/201*parcel*.txt'))
parcel_files = (parcel_files1+parcel_files2)
read_parcel = lambda path: spark.read.csv(path, header = True, sep ='|').select(sort_common_cols_2004_to_2015)
from parcel import sort_common_cols_2004_to_2015

parcel_dfs = (parcel_files
        >> map(read_parcel)
)

parcel_reduced = (parcel_dfs
        >> reduce(lambda out_df, df: out_df.union(df).distinct())
        # >> pipeable(lambda df: df.take(5))
)


# pip_parcel = (parcel_reduced
#         # >> pipeable(to_pandas)
# )
# pip_parcel

DataFrame[ACRES_DEED: string, ACRES_POLY: string, AGPRE_ENRD: string, AGPRE_EXPD: string, AG_PRESERV: string, BASEMENT: string, BLDG_NUM: string, BLOCK: string, CITY: string, CITY_USPS: string, COOLING: string, COUNTY_ID: string, DWELL_TYPE: string, EMV_BLDG: string, EMV_LAND: string, EMV_TOTAL: string, FIN_SQ_FT: string, GARAGESQFT: string, GREEN_ACRE: string, HEATING: string, HOME_STYLE: string, LANDMARK: string, LOT: string, MULTI_USES: string, NUM_UNITS: string, OPEN_SPACE: string, OWNER_MORE: string, OWNER_NAME: string, OWN_ADD_L1: string, OWN_ADD_L2: string, OWN_ADD_L3: string, PARC_CODE: string, PIN: string, PLAT_NAME: string, PREFIXTYPE: string, PREFIX_DIR: string, SALE_DATE: string, SALE_VALUE: string, SCHOOL_DST: string, SPEC_ASSES: string, STREETNAME: string, STREETTYPE: string, SUFFIX_DIR: string, Shape_Area: string, Shape_Leng: string, TAX_ADD_L1: string, TAX_ADD_L2: string, TAX_ADD_L3: string, TAX_CAPAC: string, TAX_EXEMPT: string, TAX_NAME: string, TOTAL_TAX: string, UNI

In [22]:
refselect = (ref.select('centroid_long', 'centroid_lat'))
refselect

DataFrame[centroid_long: string, centroid_lat: string]

In [15]:
pardist = parcel_reduced.drop_duplicates(['centroid_lat', 'centroid_long'])

In [16]:
joined_more = parcel_reduced.join(refselect, on = [parcel_reduced.centroid_lat == ref.centroid_lat, parcel_reduced.centroid_long == ref.centroid_long], how = "inner")

48 distinct lakes


2 missing XREF
2242335 match
44631 missing parcel

classify by distance - remember for future labs